## **Unzipping dataset in google drive:**

In [8]:
#Unzipping dataset file in google drive
!unzip "/content/drive/MyDrive/5thSemMiniProject/Drive_dataset.zip" -d "/content/drive/MyDrive/5thSemMiniProject/" 

Archive:  /content/drive/MyDrive/5thSemMiniProject/Drive_dataset.zip
   creating: /content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Training/
   creating: /content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Training/Aditya/
  inflating: /content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Training/Aditya/img1.jpg  
  inflating: /content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Training/Aditya/img10.jpg  
  inflating: /content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Training/Aditya/img100.jpg  
  inflating: /content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Training/Aditya/img1000.jpg  
  inflating: /content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Training/Aditya/img101.jpg  
  inflating: /content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Training/Aditya/img102.jpg  
  inflating: /content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Training/Aditya/img103.jpg  
  inflating: /content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Training/Aditya/img104.

### **Using MobileNet architecture for transfer learning:** 

In [9]:
from keras.applications import MobileNet

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the trained layers 
# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 ReLU False
4 DepthwiseConv2D False
5 BatchNormalization False
6 ReLU False
7 Conv2D False
8 BatchNormalization False
9 ReLU False
10 ZeroPadding2D False
11 DepthwiseConv2D False
12 BatchNormalization False
13 ReLU False
14 Conv2D False
15 BatchNormalization False
16 ReLU False
17 DepthwiseConv2D False
18 BatchNormalization False
19 ReLU False
20 Conv2D False
21 BatchNormalization False
22 ReLU False
23 ZeroPadding2D False
24 DepthwiseConv2D False
25 BatchNormalization False
26 ReLU False
27 Conv2D False
28 BatchNormalization False
29 ReLU False
30 DepthwiseConv2D False
31 BatchNormalization False
32 ReLU False
33 Conv2D False
34 BatchNormalization False
35 ReLU False
36 ZeroPadding2D False
37 DepthwiseConv2D False
38 BatchNormalization False
39 ReLU False
40 Conv2D False
41 BatchNormalization False
42 ReLU False
43 DepthwiseConv2D False
44 BatchNormalization False
45 ReLU False
46 Conv2D False
47 BatchNormalization False
48

### **Defining our neural network:**

In [10]:
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output                               # mobileNet output as the base layer
    top_model = GlobalAveragePooling2D()(top_model)               # Average pooling the last layer
    top_model = Dense(1024,activation='relu')(top_model)          # Stabilizing the model using relu activation function to 1024 neurons
    top_model = Dense(1024,activation='relu')(top_model)          # Stabilizing the model using relu activation function to 1024 neurons
    top_model = Dense(512,activation='relu')(top_model)           # Stabilizing the model using relu activation function to 512 neurons
    top_model = Dense(num_classes,activation='softmax')(top_model)# Concluding the model to 3 nodes at this layer using softmax activation function
    return top_model

In [11]:
from keras.models import Sequential                                                 #for constructing neural network
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.models import Model

# Set our class number to 2(Aditya, Anjali)
num_classes = 2

FC_Head = lw(MobileNet, num_classes)                       #building neural network

model = Model(inputs = MobileNet.input, outputs = FC_Head) #creating model

print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Training/'
validation_data_dir = '/content/drive/MyDrive/5thSemMiniProject/Drive_dataset/Validation/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(        
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')                                #defining data augmentor for training set
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')                                #defining data augmentor for validation set

Found 1995 images belonging to 2 classes.
Found 495 images belonging to 2 classes.


In [13]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("/content/drive/MyDrive/5thSemMiniProject/FaceRecogLiveVid.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 1995
nb_validation_samples = 495

# We only train 20 EPOCHS 
epochs = 20
batch_size = 32

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

<ipython-input-13-6b90c7b2e399>:33: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
62/62 [==============================] - ETA: 0s - loss: 0.3748 - accuracy: 0.9562
Epoch 1: val_loss improved from inf to 0.00000, saving model to /content/drive/MyDrive/5thSemMiniProject/FaceRecogLiveVid.h5
62/62 [==============================] - 11s 150ms/step - loss: 0.3748 - accuracy: 0.9562 - val_loss: 6.0125e-07 - val_accuracy: 1.0000
Epoch 2/20
62/62 [==============================] - ETA: 0s - loss: 3.4676e-08 - accuracy: 1.0000
Epoch 2: val_loss improved from 0.00000 to 0.00000, saving model to /content/drive/MyDrive/5thSemMiniProject/FaceRecogLiveVid.h5
62/62 [==============================] - 8s 123ms/step - loss: 3.4676e-08 - accuracy: 1.0000 - val_loss: 1.1648e-07 - val_accuracy: 1.0000
Epoch 3/20
62/62 [==============================] - ETA: 0s - loss: 6.0121e-09 - accuracy: 1.0000
Epoch 3: val_loss improved from 0.00000 to 0.00000, saving model to /content/drive/MyDrive/5thSemMiniProject/FaceRecogLiveVid.h5
62/62 [==============================] - 8s 123ms/st